In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# **1. 모두의 주차장 DB**

In [2]:
df = pd.read_csv('./data/실전db.csv', encoding='cp949')
df

,USER_ID,JOIN_DATE,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1
0,2858,2014-01-07,AA,1892,A,2020-01-01,1,GN
1,5647,2014-02-14,BB,182009,A,2020-01-01,1,J
2,33314,2014-11-20,BB,82431,A,2020-01-01,1,SC
3,37001,2014-12-04,BB,725,C,2020-01-01,1,MP
4,37819,2014-12-07,AA,220691,C,2020-01-01,1,JRR
...,...,...,...,...,...,...,...,...
879266,1830551,2020-12-31,BB,219886,B,2020-12-31,1,GN
879267,1830570,2020-12-31,BB,82433,B,2020-12-31,1,CY
879268,1830580,2020-12-31,AA,92020,B,2020-12-31,1,JRR
879269,1830589,2020-12-31,BB,92437,B,2020-12-31,1,J


**컬럼 내용을 간단하게 확인해본 결과**  
USER_ID = 사용자 ID  
JOIN_DATA = 가입일  
D_TYPE = ? (AA, BB, CC)  
STORE_ID = 상품(주차장) ID  
GOODS_TYPE = ? (A, B, C, D)  
DATE = 결제일  
COUNT = 결제건수  
AD1 = ?  

### **결측값 및 이상치**

In [3]:
df.info() # 결측값 없음 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879271 entries, 0 to 879270
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   USER_ID     879271 non-null  int64 
 1   JOIN_DATE   879271 non-null  object
 2   D_TYPE      879271 non-null  object
 3   STORE_ID    879271 non-null  int64 
 4   GOODS_TYPE  879271 non-null  object
 5   DATE        879271 non-null  object
 6   COUNT       879271 non-null  int64 
 7   AD1         879271 non-null  object
dtypes: int64(3), object(5)
memory usage: 53.7+ MB


In [4]:
# 가입일자 이상치 확인 (1970-01-01, 팀에서 논의가 필요)
np.sort(df['JOIN_DATE'].unique())

array(['1970-01-01', '2013-12-16', '2013-12-17', ..., '2020-12-29',
       '2020-12-30', '2020-12-31'], dtype=object)

In [5]:
# 결제 횟수 이상치 확인 (하루에 저렇게 결제를...?)
np.sort(df['COUNT'].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       53, 55, 58, 59, 61], dtype=int64)

### **미확인 컬럼**

In [6]:
# 이 3가지는 무엇일까?? 관련성을 찾아보자
print(df['D_TYPE'].unique())
print(df['GOODS_TYPE'].unique())
print(df['AD1'].unique())

['AA' 'BB' 'CC']
['A' 'C' 'B' 'D']
['GN' 'J' 'SC' 'MP' 'JRR' 'YD' 'GJ' 'YO' 'GSN' 'GBC' 'BF' 'DJ' '0' 'CD'
 'NO' 'CY' 'YC' 'SD' 'GD' 'GSW' 'DM' 'R' 'GB' 'GS' 'GHN' 'SB' 'IND' 'P'
 'GW' 'JR' 'GEJ' 'GGY' 'GKP' 'CAS' 'SG' 'BJG' 'BHW' 'GAY' 'GYI' 'CCA'
 'PJP' 'GGC' 'IYS' 'DB' 'GGM' 'GAS' 'WOID' 'DIV' 'AOI' 'COI' 'GPJ' 'SOI'
 'ABP' 'DOI' 'WF' 'BI' 'ZP' 'EP' 'Q' 'BO' 'ZE' 'RRR' 'QGI' 'DSO' 'SOD'
 'BOI' 'WT' 'BP' 'SP' 'CN' 'SJV' 'SDU' 'QH' 'BUD' 'CU' 'VU' 'WAE' 'NW'
 'DY' 'WH' 'GR' 'X' 'CVB' 'ADS' 'VO']


1. STORE_ID와 AD1의 관계

In [7]:
temp_df = df.groupby(['STORE_ID', 'AD1']).count().iloc[:,0].reset_index()
pd.set_option('display.max_rows', None)  # 데이터프레임 전체보기 옵션
temp_df.value_counts().sort_index()

STORE_ID  AD1   USER_ID
4         JRR   25         1
8         JRR   47         1
161       SC    5          1
179       JRR   59         1
262       MP    75         1
295       YD    3          1
353       JRR   1          1
453       GN    2          1
456       GN    6          1
466       SC    70         1
472       GN    3301       1
500       J     155        1
564       MP    54         1
607       GN    27         1
635       MP    1283       1
642       JRR   48         1
647       J     1530       1
725       MP    3058       1
774       GN    10         1
801       JRR   726        1
822       YO    160        1
943       JRR   334        1
947       JRR   1383       1
955       JRR   189        1
971       YO    7          1
978       GJ    1          1
1053      GN    197        1
1058      GN    6          1
1070      GN    325        1
1092      BF    22         1
1093      SC    61         1
1096      GD    410        1
1109      JRR   14053      1
1148      DM    36 

In [8]:
pd.set_option('display.max_rows', 20) # 전체보기 -> 20개 행 보기

각 STORE_ID마다 하나의 AD1이 매칭이 되는데, 이것은 곧 STORE_ID(지점)이 AD1이라는 구역에 속했다고 볼 수 있다. 즉 AD1은 자치구, STORE_ID는 해당 구에 있는 주차장 번호라고 해석할 수 있다.

2. D_TYPE과 GOOD_TYPE의 관계

In [9]:
df.groupby(['D_TYPE','GOODS_TYPE']).count().iloc[:,0].reset_index()

,D_TYPE,GOODS_TYPE,USER_ID
0,AA,A,345779
1,AA,B,65966
2,AA,C,70010
3,AA,D,11411
4,BB,A,262876
5,BB,B,52435
6,BB,C,60903
7,BB,D,8327
8,CC,A,1135
9,CC,B,140


D_TYPE은 AA > BB > CC 순으로 데이터가 많고  
GOODS_TYPE은 A > C > B > D 순으로 데이터가 많다. 하지만 별다른 특징은 없어보인다.  
이 부분은 조금 더 확인해 봐야겠다.

# 2. **2020교통량통합 파일**

In [10]:
# xlsx 파일보다 csv파일 속도가 더 빨라서 확장자를 csv로 변경
traffic_df = pd.read_csv("./data/2020교통량통합.csv", encoding='UTF8')
print(traffic_df.head(10))
print(traffic_df.tail(10))

         DATE        지점명  지점번호  방향            구분     0시     1시     2시     3시  \
0  2020-01-01  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  712.0  645.0  437.0  309.0   
1  2020-01-02  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  315.0  222.0  186.0  165.0   
2  2020-01-03  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  632.0  457.0  295.0  236.0   
3  2020-01-04  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  740.0  518.0  388.0  331.0   
4  2020-01-05  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  533.0  424.0  297.0  230.0   
5  2020-01-06  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  359.0  246.0  214.0  177.0   
6  2020-01-07  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  573.0  434.0  265.0  227.0   
7  2020-01-08  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  528.0  444.0  288.0  243.0   
8  2020-01-09  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  615.0  507.0  340.0  254.0   
9  2020-01-10  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  719.0  522.0  346.0  267.0   

      4시  ...     14시     15시     16시     17시     18시     19시     20시     21시  \
0  290.0  ...  1472.0  1416.0  1483.0 

- 위 자료를 직관적으로 판단해 봤을때, '방향' 컬럼에서 외부로 나가는 차량인 '유출' 데이터는 그 지역의 주차량과는 관계가 없음을 생각할 수 있다. 따라서 **유출 데이터는 삭제**해도 괜찮을듯 하다.
- '유입' 데이터 또한 그 지역을 가기 위한 차량인지 다른 곳을 가기위해 거쳐가는 단계인지 확인할 방법이 없다. 따라서 교통량 데이터는 주차와 관련이 없다고 생각하는데, 이 부분은 팀원들과 논의해볼 문제인듯하다.

# **3. 서울시 기상데이터**

In [11]:
weather_df = pd.read_csv("./data/서울시_기상데이터.csv", encoding='cp949')
weather_df

,지점,일시,기온,강수량,풍속,습도,일조시간,적설량,지면온도,지중온도
0,108,2020-01-01 1:00,-5.9,NaN,1.7,40,NaN,NaN,-2.4,3.2
1,108,2020-01-01 2:00,-5.7,NaN,0.1,42,NaN,NaN,-2.4,3.1
2,108,2020-01-01 3:00,-5.6,0.0,0.0,46,NaN,NaN,-2.7,3.1
3,108,2020-01-01 4:00,-5.4,NaN,0.0,50,NaN,NaN,-2.5,3.0
4,108,2020-01-01 5:00,-5.2,NaN,0.0,55,NaN,NaN,-2.2,3.0
...,...,...,...,...,...,...,...,...,...,...
8755,108,2020-12-30 20:00,-11.8,NaN,4.8,54,NaN,NaN,-6.5,2.2
8756,108,2020-12-30 21:00,-12.4,NaN,3.2,58,NaN,NaN,-6.9,2.2
8757,108,2020-12-30 22:00,-12.7,NaN,2.8,56,NaN,NaN,-7.1,2.1
8758,108,2020-12-30 23:00,-12.9,NaN,2.2,54,NaN,NaN,-7.3,2.1


- 일자별로 데이터가 필요 이상으로 많다. 3시간 또는 6시간 단위로 끊는것도 괜찮은 방법
- 직관적인 생각이지만 날씨별, 온도별로 주차장 이용 고객 수에 변화가 있지 않을까? 이 부분을 생각해보는 것도 괜찮겠다.

# **4. EDA**

가입 연,월,일에 따라 사용 빈도수에 차이가 있을것이다?

In [12]:
# 기존에 날짜 컬럼에서 연,월,일을 추출해 새로운 컬럼을 생성한다.
df['JOIN_YEAR'] = df['JOIN_DATE'].str.split('-', expand=True)[0]
df['JOIN_MONTH'] = df['JOIN_DATE'].str.split('-', expand=True)[1]
df['JOIN_DAY'] = df['JOIN_DATE'].str.split('-', expand=True)[2]
df

,USER_ID,JOIN_DATE,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1,JOIN_YEAR,JOIN_MONTH,JOIN_DAY
0,2858,2014-01-07,AA,1892,A,2020-01-01,1,GN,2014,01,07
1,5647,2014-02-14,BB,182009,A,2020-01-01,1,J,2014,02,14
2,33314,2014-11-20,BB,82431,A,2020-01-01,1,SC,2014,11,20
3,37001,2014-12-04,BB,725,C,2020-01-01,1,MP,2014,12,04
4,37819,2014-12-07,AA,220691,C,2020-01-01,1,JRR,2014,12,07
...,...,...,...,...,...,...,...,...,...,...,...
879266,1830551,2020-12-31,BB,219886,B,2020-12-31,1,GN,2020,12,31
879267,1830570,2020-12-31,BB,82433,B,2020-12-31,1,CY,2020,12,31
879268,1830580,2020-12-31,AA,92020,B,2020-12-31,1,JRR,2020,12,31
879269,1830589,2020-12-31,BB,92437,B,2020-12-31,1,J,2020,12,31


In [13]:
df.groupby(['JOIN_YEAR']).count().iloc[:,0]

JOIN_YEAR
1970      1093
2013        37
2014      2957
2015     14946
2016     32701
2017     79938
2018    140523
2019    257603
2020    349473
Name: USER_ID, dtype: int64

In [14]:
df.groupby(['JOIN_MONTH']).count().iloc[:,0]

JOIN_MONTH
01    67903
02    68904
03    77024
04    70403
05    70812
06    75367
07    82762
08    79618
09    76235
10    71648
11    69504
12    69091
Name: USER_ID, dtype: int64

In [15]:
df.groupby(['JOIN_DAY']).count().iloc[:,0]

JOIN_DAY
01    30555
02    30145
03    32029
04    28289
05    27935
      ...  
27    29871
28    28979
29    27555
30    26121
31    18441
Name: USER_ID, Length: 31, dtype: int64

- 가입 연도별로 나눴을 땐 최근연도로 올수록 사용 빈도가 확연히 많아졌다. 하지만 해가 거듭할수록 홍보가 되면서 많은 사람이 알게 되므로 더 많은 사람이 가입하고 사용하는것은 어쩌면 자연스러운 현상이다. 따라서 가입연도가 사용빈도수와 관련이 있다고는 할 수 없다.  
- 가입 월별로 나눴을 땐 여름(6,7,8월)이 겨울(11,12,1,2월)에 비해 사용 빈도수가 많음을 알 수 있다. 이것은 분명 의미가 있는 데이터이고, 기상 데이터에서 '온도'와 비교해도 괜찮은 근거를 갖게 해준다.  
- 가입 일자별로 나눴을 땐 빈도수에 편차가 있긴 하지만 특정 구역에서 많고 적음이 아닌 하루 차이로 들쭉날쭉한 모습을 보여준다. 따라서 가입일자는 사용빈도수와 관련이 있다고 할 수 없다.  
  
**날씨가 더울때(여름)가 추울때(겨울)보다 사용빈도수가 더 많음을 알았으니 기상데이터로 확인해보자**